In [205]:
import pandas as pd
 
df = pd.read_excel('MVO Assignment.xlsx')
tickers = df['Unnamed: 1'].dropna()
param = list(df['Unnamed: 4'].dropna())


In [206]:
import yfinance as yf

history = pd.DataFrame()

for ticker in tickers:
    yf_ticker = yf.Ticker(ticker)
    history[ticker] = yf_ticker.history(period=param[0], interval = param[1], auto_adjust = True)['Close']

    col_shift = ticker + '_shifted'
    history[col_shift] = history[ticker].shift(periods = 1,fill_value = 0, axis = 0)

    col_ret = ticker + '_returns'
    history[col_ret] = (history[col_shift] - history[ticker])/history[col_shift]
    
    history = history.drop(columns=[col_shift, ticker])
    
history.index = history.index.tz_localize(None)


print(history)



            AAPL_returns  WMT_returns  QQQ_returns  CAT_returns
Date                                                           
2013-09-01          -inf         -inf         -inf         -inf
2013-10-01     -0.096382    -0.037723    -0.052721     0.000480
2013-11-01     -0.063842    -0.055505    -0.035511    -0.021983
2013-12-01     -0.014792     0.028638    -0.026012    -0.073404
2014-01-01      0.107697     0.045435     0.016098    -0.034138
...                  ...          ...          ...          ...
2023-04-01     -0.028987    -0.028069    -0.006635     0.043873
2023-05-01     -0.044613     0.027158    -0.078838     0.054539
2023-06-01     -0.095844    -0.074263    -0.061582    -0.195869
2023-07-01     -0.012785    -0.017051    -0.040026    -0.077708
2023-08-01      0.019700     0.004003     0.024213    -0.089845

[120 rows x 4 columns]


In [207]:
#history.tail(-1).to_excel(r'MVO Assignment.xlsx', sheet_name='Returns', index=True)


In [208]:
import openpyxl
wb = openpyxl.load_workbook('MVO Assignment.xlsx', data_only=True)
sheet = wb['Returns']

In [214]:
sheet.delete_cols(1, 100000)
sheet.delete_rows(1, 100000)


columns = history.columns.values.tolist()
sheet.cell(column=1, row=1).value = 'Time'



for i, time in enumerate(history.index.values.tolist()):
    if i==0:
        pass
    else:
        sheet.cell(column=1, row=i+1).value = pd.to_datetime(time)


for i, column in enumerate(columns):
    
    for j, value in enumerate(history[column].tolist()):
        sheet.cell(column=i+2, row=j+1).value = value
        
for i, column in enumerate(columns):
    sheet.cell(column=i+2, row=1).value = column

wb.save('MVO Assignment.xlsx')

In [216]:
import numpy as np

np.matmul(np.transpose(history.tail(-1)),history.tail(-1))/

/var/folders/hg/kqg69xz568l28vnvcx6x09hh0000gn/T/ipykernel_94748/1659715124.py:3: FutureWarning: Calling a ufunc on non-aligned DataFrames (or DataFrame/Series combination). Currently, the indices are ignored and the result takes the index/columns of the first DataFrame. In the future , the DataFrames/Series will be aligned before applying the ufunc.
Convert one of the arguments to a NumPy array (eg 'ufunc(df1, np.asarray(df2)') to keep the current behaviour, or align manually (eg 'df1, df2 = df1.align(df2)') before passing to the ufunc to obtain the future behaviour and silence this warning.
  np.matmul(np.transpose(history.tail(-1)),history.tail(-1))


,0,1,2,3
AAPL_returns,0.819747,0.195562,0.437924,0.273913
WMT_returns,0.195562,0.340039,0.132255,0.130475
QQQ_returns,0.437924,0.132255,0.357493,0.264177
CAT_returns,0.273913,0.130475,0.264177,0.820408
